-sandbox
<img width = '500' src="https://databricks.com/wp-content/uploads/2021/06/db-pride-logo.svg" style="float: left: margin: 30px"/>        <img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 10px"/>

<br>
> # Raw para Bronze 

<br>
## 05 - Dados Macro

Processamento de dados externos para apoiar nossa tomada de decisão

Uma das fontes são os Dados Macroeconômicos sem conexão com nossos conjuntos de dados internos: recebemos esses dados todos os meses do banco central, contendo indicadores macroeconômicos como desemprego e o PIB
  
Isso ajuda a construir uma única métrica decimal, que é usada para atualizar as pontuações de crédito dos investidores. Essas pontuações de crédito servem como base para a aprovação ou negação de solicitações de empréstimo


## Objetivos:
* Crie uma tabela Delta a partir do CSV de dados de macro
* Defina o esquema e grave o DataFrame no diretório especificado
* Definir um esquema CSV manualmente

-sandbox
<h2 style="color:red">Informações:</h2>


- esta é a última tabela bronze
- usar dados de terceiros para atualizar métricas é uma prática comum
- vantagens de importar esses dados no data lake
- diferentes opções de definição de esquema, incluindo DDL

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/06-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

...

Cleaning up the learning environment...no actions taken.

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 macroDataPath : dbfs:/mnt/training/finance-org/macro/macro.csv 
 dbfs:/mnt/training/finance-org/macro/macro.csv 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/05_dados_macro_psp/macro.delta 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/05_dados_macro_psp/macro.delta 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_05_dados_macro_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

Declared the following function:
 realityCheck

All done!

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas: 


<br>
- Explore o esquema do arquivo CSV: use o comando **`%fs head`** em uma nova célula. <a href="https://docs.databricks.com/dev-tools/databricks-utils.html#dbutils-fs" target="_blank"> Documentação </a>
- Leia o arquivo CSV de fornecedores de **`macroDataPath`** em um DataFrame
- Defina o esquema de acordo com a tabela abaixo
- Grave o DataFrame final como uma tabela Delta em **`targetDirectory`**
- Retorne o DataFrame final, que deve ter o seguinte esquema:

|name|type|
|---|---|
|date|DateType|
|fedfunds|DoubleType|
|gdp|FloatType|
|bamlh0a0hym2|FloatType|
|gfdegdq188s|FloatType|
|indpro|FloatType|
|a191rl1q225sbea|FloatType|
|indgs10dpro|FloatType|
|unrate|FloatType|
|dexuseu|FloatType|

In [0]:
%fs head dbfs:/mnt/training/finance-org/macro/macro.csv

[Truncated to first 65536 bytes]
date;fedfunds;gdp;bamlh0a0hym2;gfdegdq188s;indpro;a191rl1q225sbea;dgs10;unrate;dexuseu
2010-01-01;0.18;14721.35;6.85;86.76598;91.6642;3.7;3.62;9.8;1.3699
2010-02-01;0.18;14721.35;6.85;86.76598;91.994;3.7;3.62;9.8;1.3699
2010-02-08;0.18;14721.35;6.85;86.76598;91.994;3.7;3.62;9.8;1.3699
2010-02-09;0.18;14721.35;6.88;86.76598;91.994;3.7;3.67;9.8;1.3795
2010-02-10;0.18;14721.35;6.93;86.76598;91.994;3.7;3.72;9.8;1.3722
2010-02-11;0.18;14721.35;6.98;86.76598;91.994;3.7;3.73;9.8;1.3640
2010-02-12;0.18;14721.35;7.03;86.76598;91.994;3.7;3.69;9.8;1.3625
2010-02-15;0.18;14721.35;7.01;86.76598;91.994;3.7;.;9.8;.
2010-02-16;0.18;14721.35;7.02;86.76598;91.994;3.7;3.66;9.8;1.3742
2010-02-17;0.18;14721.35;6.80;86.76598;91.994;3.7;3.74;9.8;1.3623
2010-02-18;0.18;14721.35;6.65;86.76598;91.994;3.7;3.79;9.8;1.3563
2010-02-19;0.18;14721.35;6.56;86.76598;91.994;3.7;3.78;9.8;1.3538
2010-02-22;0.18;14721.35;6.50;86.76598;91.994;3.7;3.80;9.8;1.3590
2010-02-23;0.18;14721.35;6.60;86.76598;91.994;3.7;3.69;9.8;1.3537
2010-02-24;0.18;14721.35;6.58;86.76598;91.994;3.7;3.70;9.8;1.3576
2010-02-25;0.18;14721.35;6.65;86.76598;91.994;3.7;3.64;9.8;1.3476
2010-02-26;0.18;14721.35;6.66;86.76598;91.994;3.7;3.61;9.8;1.3660
2010-02-28;0.18;14721.35;6.71;86.76598;91.994;3.7;3.61;9.8;1.3660
2010-03-01;0.18;14721.35;6.66;86.76598;92.5993;3.7;3.61;9.9;1.3516
2010-03-02;0.18;14721.35;6.58;86.76598;92.5993;3.7;3.62;9.9;1.3564
2010-03-03;0.18;14721.35;6.54;86.76598;92.5993;3.7;3.63;9.9;1.3731
2010-03-04;0.18;14721.35;6.49;86.76598;92.5993;3.7;3.61;9.9;1.3571
2010-03-05;0.18;14721.35;6.37;86.76598;92.5993;3.7;3.69;9.9;1.3608
2010-03-08;0.18;14721.35;6.27;86.76598;92.5993;3.7;3.72;9.9;1.3613
2010-03-09;0.18;14721.35;6.23;86.76598;92.5993;3.7;3.71;9.9;1.3586
2010-03-10;0.18;14721.35;6.16;86.76598;92.5993;3.7;3.73;9.9;1.3658
2010-03-11;0.18;14721.35;6.15;86.76598;92.5993;3.7;3.73;9.9;1.3674
2010-03-12;0.18;14721.35;6.11;86.76598;92.5993;3.7;3.71;9.9;1.3753
2010-03-15;0.18;14721.35;6.12;86.76598;92.5993;3.7;3.71;9.9;1.3652
2010-03-16;0.18;14721.35;6.16;86.76598;92.5993;3.7;3.66;9.9;1.3758
2010-03-17;0.18;14721.35;6.11;86.76598;92.5993;3.7;3.65;9.9;1.3739
2010-03-18;0.18;14721.35;6.02;86.76598;92.5993;3.7;3.68;9.9;1.3603
2010-03-19;0.18;14721.35;5.98;86.76598;92.5993;3.7;3.70;9.9;1.3530
2010-03-22;0.18;14721.35;6.02;86.76598;92.5993;3.7;3.67;9.9;1.3531
2010-03-23;0.18;14721.35;6.00;86.76598;92.5993;3.7;3.69;9.9;1.3535
2010-03-24;0.18;14721.35;5.85;86.76598;92.5993;3.7;3.84;9.9;1.3347
2010-03-25;0.18;14721.35;5.78;86.76598;92.5993;3.7;3.91;9.9;1.3344
2010-03-26;0.18;14721.35;5.80;86.76598;92.5993;3.7;3.86;9.9;1.3398
2010-03-29;0.18;14721.35;5.79;86.76598;92.5993;3.7;3.88;9.9;1.3465
2010-03-30;0.18;14721.35;5.77;86.76598;92.5993;3.7;3.88;9.9;1.3409
2010-03-31;0.18;14721.35;5.84;86.76598;92.5993;3.7;3.84;9.9;1.3526
2010-04-01;0.18;14926.098;5.82;88.44771;92.9436;3.7;3.89;9.9;1.3569
2010-04-02;0.18;14926.098;.;88.44771;92.9436;3.7;3.96;9.9;1.3487
2010-04-05;0.18;14926.098;5.65;88.44771;92.9436;3.7;4.01;9.9;1.3486
2010-04-06;0.18;14926.098;5.65;88.44771;92.9436;3.7;3.98;9.9;1.3378
2010-04-07;0.18;14926.098;5.72;88.44771;92.9436;3.7;3.89;9.9;1.3364
2010-04-08;0.18;14926.098;5.70;88.44771;92.9436;3.7;3.91;9.9;1.3360
2010-04-09;0.18;14926.098;5.68;88.44771;92.9436;3.7;3.90;9.9;1.3468
2010-04-12;0.18;14926.098;5.68;88.44771;92.9436;3.7;3.87;9.9;1.3587
2010-04-13;0.18;14926.098;5.66;88.44771;92.9436;3.7;3.84;9.9;1.3592
2010-04-14;0.18;14926.098;5.57;88.44771;92.9436;3.7;3.88;9.9;1.3666
2010-04-15;0.18;14926.098;5.52;88.44771;92.9436;3.7;3.86;9.9;1.3540
2010-04-16;0.18;14926.098;5.57;88.44771;92.9436;3.7;3.79;9.9;1.3487
2010-04-19;0.18;14926.098;5.59;88.44771;92.9436;3.7;3.83;9.9;1.3458
2010-04-20;0.18;14926.098;5.54;88.44771;92.9436;3.7;3.82;9.9;1.3446
2010-04-21;0.18;14926.098;5.55;88.44771;92.9436;3.7;3.77;9.9;1.3383
2010-04-22;0.18;14926.098;5.50;88.44771;92.9436;3.7;3.80;9.9;1.3298
2010-04-23;0.18;14926.098;5.44;88.44771;92.9436;3.7;3.84;9.9;1.3360
2010-

In [0]:
def func_DadosMacro(spark, macroDataPath, targetDirectory):
  
  
  from pyspark.sql.functions import col
  from pyspark.sql.types import IntegerType
  
  
  # Defina o esquema de acordo com as "Etapas a serem concluídas"
  macroSchema = """ date DATE, 
                    fedfunds DOUBLE, 
                    gdp FLOAT, 
                    bamlh0a0hym2 FLOAT, 
                    gfdegdq188s FLOAT, 
                    indpro FLOAT, 
                    a191rl1q225sbea FLOAT, 
                    dgs10 FLOAT, 
                    unrate FLOAT, 
                    dexuseu FLOAT
                """
  
  
  # Leia macroDataPath e certifique-se de que:
  # a primeira linha é o cabeçalho e
  # o delimitador correto é especificado.
  df = (spark.read
             .schema(macroSchema)
             .option("header", True)
             .option("delimiter", ";")
             .csv(macroDataPath)
             .dropna())
  

  # Grave o DataFrame final como uma tabela Delta em targetDirectory
  # Certifique-se de deixar o Spark sobrescrever o local de destino
  (df.write
     .mode("overwrite")
     .option("overwriteSchema", "true")
     .format("delta")
     .save(targetDirectory))
  

  return df

In [0]:
dfFinal = func_DadosMacro(spark, macroDataPath, targetDirectory)

In [0]:
display(dfFinal)

date,fedfunds,gdp,bamlh0a0hym2,gfdegdq188s,indpro,a191rl1q225sbea,dgs10,unrate,dexuseu
2010-01-01,0.18,14721.35,6.85,86.76598,91.6642,3.7,3.62,9.8,1.3699
2010-02-01,0.18,14721.35,6.85,86.76598,91.994,3.7,3.62,9.8,1.3699
2010-02-08,0.18,14721.35,6.85,86.76598,91.994,3.7,3.62,9.8,1.3699
2010-02-09,0.18,14721.35,6.88,86.76598,91.994,3.7,3.67,9.8,1.3795
2010-02-10,0.18,14721.35,6.93,86.76598,91.994,3.7,3.72,9.8,1.3722
2010-02-11,0.18,14721.35,6.98,86.76598,91.994,3.7,3.73,9.8,1.364
2010-02-12,0.18,14721.35,7.03,86.76598,91.994,3.7,3.69,9.8,1.3625
2010-02-16,0.18,14721.35,7.02,86.76598,91.994,3.7,3.66,9.8,1.3742
2010-02-17,0.18,14721.35,6.8,86.76598,91.994,3.7,3.74,9.8,1.3623
2010-02-18,0.18,14721.35,6.65,86.76598,91.994,3.7,3.79,9.8,1.3563


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Qualidade dos dados</h2>

In [0]:


assert \
      (spark.read
           .schema(""" date DATE, 
                       fedfunds DOUBLE, 
                       gdp FLOAT, 
                       bamlh0a0hym2 FLOAT, 
                       gfdegdq188s FLOAT, 
                       indpro FLOAT, 
                       a191rl1q225sbea FLOAT, 
                       dgs10 FLOAT, 
                       unrate FLOAT, 
                       dexuseu FLOAT
                   """)
            .option("delimiter", ";")
            .option("header", True)
            .csv(macroDataPath)
            .dropna()
            .count()) \
== \
      (dfFinal.count()) \
, displayHTML("""<font size="5" color="red" face="sans-serif">Validação: falhou !!!</font>""")

displayHTML("""<font size="5" color="green" face="sans-serif">Validação: correta !!!</font>""")

Validação: correta !!!

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, FloatType, DateType

schemaOriginal = (StructType([
                               StructField("date"             ,DateType()    ,True),
                               StructField("fedfunds"         ,DoubleType()  ,True),
                               StructField("gdp"              ,FloatType()   ,True),
                               StructField("bamlh0a0hym2"     ,FloatType()   ,True),
                               StructField("gfdegdq188s"      ,FloatType()   ,True),
                               StructField("indpro"           ,FloatType()   ,True),
                               StructField("a191rl1q225sbea"  ,FloatType()   ,True),
                               StructField("dgs10"            ,FloatType()   ,True),
                               StructField("unrate"           ,FloatType()   ,True),
                               StructField("dexuseu"          ,FloatType()   ,True)
                              ]))

In [0]:
assert dfFinal.schema == schemaOriginal, displayHTML("""<font size="5" color="red" face="sans-serif">Validação: falhou !!!</font>""")

displayHTML("""<font size="5" color="green" face="sans-serif">Validação: correta !!!</font>""")

Validação: correta !!!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_DadosMacro, spark, macroDataPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Delta table in place,
1,Delta table has correct content,


d
# <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Fim do pipeline de ingestão na Bronze

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>